In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
zhfont = FontProperties(fname="/System/Library/Fonts/PingFang.ttc")
import numpy as np
import tushare as ts 
token = "a7c1312fe8187bff67ada6333c4b447caff18f58be66ac8b64d20d5a"
api = ts.pro_api(token)

In [131]:
import os

In [2]:
df = pd.read_csv("both.csv")

In [3]:
df

,code,name,mark,startTime_roe,check,industry,startTime_cf,check.1,both
0,000799.SZ,酒鬼酒,I,2014-12,1.0,食品饮料,2020-03,1.0,1.0
1,002216.SZ,三全食品,I,2017-03,1.0,食品饮料,2019-12,1.0,1.0
2,002568.SZ,百润股份,I,2017-06,1.0,食品饮料,2017-09,1.0,1.0
3,002661.SZ,克明面业,I,2018-06,1.0,食品饮料,2018-06,1.0,1.0
4,600809.SH,山西汾酒,I,2015-09,1.0,食品饮料,2018-12,1.0,1.0
...,...,...,...,...,...,...,...,...,...
106,002080.SZ,中材科技,I,2017-09,1.0,化工,2017-06,1.0,1.0
107,002556.SZ,辉隆股份,I,2019-12,1.0,化工,2018-12,1.0,1.0
108,002683.SZ,宏大爆破,I,2016-12,1.0,化工,2016-12,1.0,1.0
109,002194.SZ,武汉凡谷,I,2018-06,1.0,通信,2018-12,1.0,1.0


In [7]:
codes = df.code.values
stRoe = df.startTime_roe.values
stCf = df.startTime_cf.values

## 测试
- pearson

- granger

In [38]:
tempCode = codes[0]
roeTime = stRoe[0]
cfTime = stCf[0]
st1 = "".join(roeTime.split("-"))+"01"  # start time roe
st2 = "".join(cfTime.split("-"))+"01"  # start time cf
et = "20201231"  # end time

In [123]:
price = ts.pro_bar(ts_code=tempCode, adj='qfq', start_date=st1, end_date=et)

In [132]:
price.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000799.SZ,2020-12-31,154.51,158.80,151.86,156.50,153.99,2.51,1.6300,135903.94,2112010.281
1,000799.SZ,2020-12-30,144.16,154.78,143.50,153.99,144.88,9.11,6.2880,167853.06,2518328.321
2,000799.SZ,2020-12-29,145.00,146.68,139.27,144.88,140.34,4.54,3.2350,198647.06,2850951.003
3,000799.SZ,2020-12-28,133.00,140.34,133.00,140.34,127.58,12.76,10.0016,118251.86,1646735.579
4,000799.SZ,2020-12-25,124.43,128.88,122.43,127.58,125.43,2.15,1.7141,95324.46,1204740.228


In [17]:
price["trade_date"] = pd.to_datetime(price["trade_date"])

In [20]:
quarterly = price.resample("Q", on="trade_date").mean()

In [31]:
roe = pd.read_csv("111家roe数据.csv")
cf = pd.read_csv("111家cf数据.csv")

In [33]:
tempRoe = roe[roe["code"]==tempCode]
tempCf = cf[cf["code"]==cf]

In [46]:
t = tempRoe.columns.get_indexer([roeTime,])[0]

In [48]:
roeTime

'2014-12'

In [52]:
# tempRoe.iloc[0,t:]  # 最后两个时间错位是concat的问题 Mar2中

In [53]:
roeData = tempRoe.iloc[0, t:]

In [56]:
roeData.index = pd.to_datetime(roeData.index)

In [62]:
tt = roeData.resample("Q").last()

In [65]:
ttt = tt.dropna()

In [76]:
ttt.index[-1]

Timestamp('2020-09-30 00:00:00', freq='Q-DEC')

In [71]:
qPrice = quarterly["close"]

In [86]:
tttt = pd.DataFrame({"roe":ttt, "close":qPrice[:ttt.index[-1]]})

In [105]:
tttt["roe"] = tttt["roe"].apply(pd.to_numeric)

In [110]:
tttt["roe"].apply(pd.to_numeric).corr(qPrice[:ttt.index[-1]])

0.417899533878764

In [144]:
tttt["roe"].apply(pd.to_numeric).corr(qPrice[:ttt.index[-1]],method="spearman")

0.2381422924901186

In [145]:
from scipy.stats import spearmanr

In [149]:
tttt["roe"] = tttt["roe"].apply(pd.to_numeric)

In [150]:
temp1 = tttt.dropna()

In [151]:
spearmanr(temp1["roe"], temp1["close"])

SpearmanrResult(correlation=0.2381422924901186, pvalue=0.2738506728809402)

In [115]:
pd.to_numeric(ttt)

2014-12-31   -16.9000
2015-03-31    14.8769
2015-06-30    31.3040
2015-09-30    47.3355
2015-12-31    38.3659
2016-03-31    36.5277
2016-06-30    28.9937
2016-09-30    22.1432
2016-12-31    33.4898
2017-03-31    24.6915
2017-06-30    22.8455
2017-09-30    22.0153
2017-12-31    25.6306
2018-03-31    25.8622
2018-06-30    29.3604
2018-09-30    21.3855
2018-12-31    17.8835
2019-03-31    16.3536
2019-06-30    18.8966
2019-09-30    23.9583
2019-12-31    25.2602
2020-03-31    23.0045
2020-06-30    32.3351
2020-09-30    43.2680
Freq: Q-DEC, Name: 0, dtype: float64

## 检验Pearson相关系数

In [218]:
def correlate(startTime:str, df:pd.DataFrame, price:pd.DataFrame)->float:
    '''
    Description:
    根据startTime从df和price中获取开始时间之后的季度数据
    返回pearson相关系数和spearman相关系数及其pvalue
    ---
    Params:
    startTime, str, like "2014-01"
    df, DataFrame of only one row, company roe or cf data
    price, DataFrame, tushare 请求返回数据
    startTime, str, like 2014-01
    ---
    Returns:
    corrCoef, float 
    '''
    # 获取开始时间所在的下标
    timeIdx = df.columns.get_indexer([startTime,])[0]
    data = df.iloc[0, timeIdx:]
    data.index = pd.to_datetime(data.index)
    # 转化成季度数据
    data = data.resample("Q").last()
    data.dropna(inplace=True)
    endTime = data.index[-1]
    # 获取价格的quarterly数据
    price["trade_date"] = pd.to_datetime(price["trade_date"], format="%Y%m%d")
    quarterly = price.resample("Q", on="trade_date").mean()
    price = quarterly[:endTime]["close"]
    temp = pd.DataFrame(data={"roe":pd.to_numeric(data), "close":price})
    temp.dropna(inplace=True)
    pearson = temp["roe"].corr(temp["close"])
    spearman = spearmanr(temp["roe"], temp["close"])
    return pearson, spearman

In [156]:
pearson, spearman = correlate(roeTime, tempRoe, price)

In [160]:
pearson, spearman[0], spearman[1]

(0.417899533878764, 0.2381422924901186, 0.2738506728809402)

In [170]:
def getPriceData(code:str, startTime:str, endTime:str = "20201231", saveBool:bool =True):
    '''
    Description:
    根据code时间等从tushare获取股价信息
    ---
    Params:
    code, str
    startTime, str, like 2014-01
    endTime, str, like 20201231
    saveBool, bool, whether to save data as csv file, outfile named by code and starttime
    ---
    Returns:
    price
    '''
    outpath = f"data/{code[:-3]}-{startTime}.csv"
    if os.path.exists(outpath):
        # print("downloaded in", outpath)
        price = pd.read_csv(outpath)
        return price

    st = "".join(startTime.split("-"))+"01"
    price = ts.pro_bar(ts_code=code, adj='qfq', start_date=st, end_date=endTime)
    if saveBool:
        if not os.path.exists("data/"):
            os.mkdir("data/")
        price.to_csv(outpath, index=False)
    return price

In [137]:
# getPriceData(tempCode, "2014-01")

In [143]:
def getEarlyTime(time1:str, time2:str):
    '''
    Description:
    获取两个时间中较早的并返回
    ---
    Params:
    time, str, like 2014-01
    ---
    Returns:
    time, str, like 2014-01
    '''
    return time1 if time1<time2 else time2

In [161]:
# codes = df.code.values
# stRoe = df.startTime_roe.values
# stCf = df.startTime_cf.values
# df 

In [162]:
df.head()

,code,name,mark,startTime_roe,check,industry,startTime_cf,check.1,both
0,000799.SZ,酒鬼酒,I,2014-12,1.0,食品饮料,2020-03,1.0,1.0
1,002216.SZ,三全食品,I,2017-03,1.0,食品饮料,2019-12,1.0,1.0
2,002568.SZ,百润股份,I,2017-06,1.0,食品饮料,2017-09,1.0,1.0
3,002661.SZ,克明面业,I,2018-06,1.0,食品饮料,2018-06,1.0,1.0
4,600809.SH,山西汾酒,I,2015-09,1.0,食品饮料,2018-12,1.0,1.0


## test

In [172]:
count = 1
for code, startTime_roe, startTime_cf in zip(codes, stRoe, stCf):
    print(code, startTime_roe, startTime_cf)
    count += 1
    if count ==6:
        break

000799.SZ 2014-12 2020-03
002216.SZ 2017-03 2019-12
002568.SZ 2017-06 2017-09
002661.SZ 2018-06 2018-06
600809.SH 2015-09 2018-12


In [165]:
import time

In [206]:
code = "000799.SZ"
startTime_roe = "2014-12"
startTime_cf = "2020-03"
earlyTime = getEarlyTime(startTime_roe, startTime_cf)
print(earlyTime)
price = getPriceData(code, earlyTime, saveBool=True)
tempRoe = roe[roe["code"]==code]
tempCf = cf[cf["code"]==code]

2014-12


In [191]:
# price

In [180]:
timeIdx = tempRoe.columns.get_indexer([startTime_roe, ])[0]
timeIdx

59

In [181]:
data = tempRoe.iloc[0, timeIdx:]

In [182]:
data.index = pd.to_datetime(data.index)

In [183]:
data = data.resample("Q").last()
data.dropna(inplace=True)

In [185]:
endTime = data.index[-1]

In [207]:
# 获取价格的quarterly数据
price["trade_date"] = pd.to_datetime(price["trade_date"], format="%Y%m%d")


In [209]:
quarterly = price.resample("Q", on="trade_date").mean()


In [211]:
price1 = quarterly[:endTime]["close"]

In [214]:
temp = pd.DataFrame(data={"roe":pd.to_numeric(data), "close":price1})
temp.dropna(inplace=True)
pearson = temp["roe"].corr(temp["close"])
spearman = spearmanr(temp["roe"], temp["close"])

In [217]:
pearson, spearman

(0.417899533878764,
 SpearmanrResult(correlation=0.2381422924901186, pvalue=0.2738506728809402))

In [219]:
correlate(startTime_roe, tempRoe, price)

(0.417899533878764,
 SpearmanrResult(correlation=0.2381422924901186, pvalue=0.2738506728809402))

## main

In [221]:
roePearson = {}
roeSpearman = {}
cfPearson = {}
cfSpearman = {}
failed = []
for code, startTime_roe, startTime_cf in zip(codes, stRoe, stCf):
    print(code, end=" ")
    try:
        earlyTime = getEarlyTime(startTime_roe, startTime_cf)
        price = getPriceData(code, earlyTime, saveBool=True)
        tempRoe = roe[roe["code"]==code]
        tempCf = cf[cf["code"]==code]
    except Exception as e:
        print(e)
        failed.append((code, startTime_roe, startTime_cf))
        time.sleep(2)
        continue
    roePear, roeSpear = correlate(startTime_roe, tempRoe, price)
    cfPear, cfSpear = correlate(startTime_cf, tempCf, price)
    roePearson[code] = roePear
    roeSpearman[code] = roeSpear
    cfPearson[code] = cfPear
    cfSpearman[code] = cfSpear
    time.sleep(1)
    print(roePear, roeSpear)

000799.SZ 0.417899533878764 SpearmanrResult(correlation=0.2381422924901186, pvalue=0.2738506728809402)
002216.SZ 0.9226115992931546 SpearmanrResult(correlation=0.4142857142857142, pvalue=0.12471544223657098)
002568.SZ 0.6291239162073572 SpearmanrResult(correlation=0.5736263736263736, pvalue=0.03197081397639698)
002661.SZ 0.6237490983718676 SpearmanrResult(correlation=0.34545454545454546, pvalue=0.32822651147136733)
600809.SH 0.48064535050090434 SpearmanrResult(correlation=0.49870129870129876, pvalue=0.02138084599925308)
600866.SH 0.7166428438920429 SpearmanrResult(correlation=0.6833333333333333, pvalue=0.04244227208707231)
600872.SH 0.5192147337336948 SpearmanrResult(correlation=0.5614035087719299, pvalue=0.012380476816162053)
000682.SZ 0.6342684613974394 SpearmanrResult(correlation=0.5272727272727272, pvalue=0.11730806555020223)
002129.SZ 0.6173061515421819 SpearmanrResult(correlation=0.42499999999999993, pvalue=0.11429547548052224)
002576.SZ 0.16901512469443636 SpearmanrResult(correl

In [224]:
len(codes)

111

In [228]:
roe1 = []  # Pearson
roe2 = []  # Spearman
roe3 = []  # pvalue for Spearman
cf1 = []
cf2 = []
cf3 = []
tempCodes = list(roePearson.keys())
for code in tempCodes:
    roe1.append(roePearson[code])
    roe2.append(roeSpearman[code][0])
    roe3.append(roeSpearman[code][1])
    cf1.append(cfPearson[code])
    cf2.append(cfSpearman[code][0])
    cf3.append(cfSpearman[code][1])

In [229]:
failed  # 无数据

[('200019.SZ', '2018-12', '2019-03'),
 ('200625.SZ', '2020-03', '2019-06'),
 ('200512.SZ', '2018-12', '2019-03'),
 ('900911.SH', '2016-09', '2020-03'),
 ('900939.SH', '2019-03', '2017-09')]

In [247]:
names = []
for code in tempCodes:
    names.append(df[df["code"]==code].name.values[0])

In [249]:
checkDf = pd.DataFrame(data={"code":tempCodes, "name":names, 
                             "roe_Pearson":roe1, "roe_Spearman":roe2, "roe_pvalue":roe3,
                             "cf_Pearson":cf1, "cf_Spearman":cf2, "cf_pvalue":cf3})

In [251]:
checkDf.to_csv("roe-cf-price相关系数.csv", index=False)

## 筛选公司
- 从相关系数筛选
- 看p值

In [252]:
checkDf.head()

,code,name,roe_Pearson,roe_Spearman,roe_pvalue,cf_Pearson,cf_Spearman,cf_pvalue
0,000799.SZ,酒鬼酒,0.417900,0.238142,0.273851,0.939209,0.500000,0.666667
1,002216.SZ,三全食品,0.922612,0.414286,0.124715,-0.222069,-0.400000,0.600000
2,002568.SZ,百润股份,0.629124,0.573626,0.031971,0.844735,0.829670,0.000450
3,002661.SZ,克明面业,0.623749,0.345455,0.328227,0.380465,0.296970,0.404702
4,600809.SH,山西汾酒,0.480645,0.498701,0.021381,0.140199,0.190476,0.651401


In [256]:
len(checkDf[checkDf["roe_pvalue"]<0.1])

34

In [257]:
len(checkDf[checkDf["cf_pvalue"]<0.1])

25

In [258]:
checkDf[(checkDf["roe_pvalue"]<0.1) & (checkDf["cf_pvalue"]<0.1)]

,code,name,roe_Pearson,roe_Spearman,roe_pvalue,cf_Pearson,cf_Spearman,cf_pvalue
2,002568.SZ,百润股份,0.629124,0.573626,0.031971,0.844735,0.829670,0.000450
5,600866.SH,星湖科技,0.716643,0.683333,0.042442,-0.544733,-0.535797,0.005772
27,601100.SH,恒立液压,0.726252,0.897059,0.000001,0.599403,0.590909,0.055576
49,002409.SZ,雅克科技,0.702850,0.766667,0.015944,0.738777,0.933333,0.000236
51,002463.SZ,沪电股份,0.772618,0.724675,0.000203,0.746098,0.644295,0.003901
75,002312.SZ,三泰控股,0.857284,0.900000,0.037386,0.790852,0.672727,0.033041
86,000407.SZ,胜利股份,-0.572053,-0.780220,0.000994,-0.305935,-0.389231,0.054462
92,000661.SZ,长春高新,0.439242,0.536364,0.088953,0.774100,0.800000,0.003110
93,000739.SZ,普洛药业,0.657358,0.854545,0.001637,0.728386,0.776224,0.002993
103,002683.SZ,宏大爆破,0.398804,0.611765,0.011788,0.444148,0.597059,0.014610


In [267]:
df[df["code"]=="600866.SH"]

,code,name,mark,startTime_roe,check,industry,startTime_cf,check.1,both
5,600866.SH,星湖科技,I,2018-09,1.0,食品饮料,2014-03,1.0,1.0


## 去除负相关的

In [270]:
positive = checkDf[(checkDf["roe_Pearson"]>0)&(checkDf["cf_Pearson"]>0)&(checkDf["roe_Spearman"]>0)&(checkDf["cf_Spearman"]>0)]

In [271]:
len(checkDf), len(positive)

(106, 53)

In [273]:
len(positive[positive["roe_pvalue"]<0.1])

19

In [274]:
len(positive[positive["cf_pvalue"]<0.1])

17

In [275]:
positive[(positive["cf_pvalue"]<0.1)&(positive["roe_pvalue"]<0.1)]

,code,name,roe_Pearson,roe_Spearman,roe_pvalue,cf_Pearson,cf_Spearman,cf_pvalue
2,002568.SZ,百润股份,0.629124,0.573626,0.031971,0.844735,0.829670,0.000450
27,601100.SH,恒立液压,0.726252,0.897059,0.000001,0.599403,0.590909,0.055576
49,002409.SZ,雅克科技,0.702850,0.766667,0.015944,0.738777,0.933333,0.000236
51,002463.SZ,沪电股份,0.772618,0.724675,0.000203,0.746098,0.644295,0.003901
75,002312.SZ,三泰控股,0.857284,0.900000,0.037386,0.790852,0.672727,0.033041
92,000661.SZ,长春高新,0.439242,0.536364,0.088953,0.774100,0.800000,0.003110
93,000739.SZ,普洛药业,0.657358,0.854545,0.001637,0.728386,0.776224,0.002993
103,002683.SZ,宏大爆破,0.398804,0.611765,0.011788,0.444148,0.597059,0.014610


In [276]:
positive[positive["roe_Pearson"]>0.7]

,code,name,roe_Pearson,roe_Spearman,roe_pvalue,cf_Pearson,cf_Spearman,cf_pvalue
14,002014.SZ,永新股份,0.851215,0.818182,0.001143,0.354754,0.350000,0.355820
17,002111.SZ,威海广泰,0.728042,0.538462,0.070894,0.186173,0.238095,0.570156
22,300171.SZ,东富龙,0.853513,0.900000,0.037386,0.661613,0.443771,0.198893
26,601038.SH,一拖股份,0.889649,0.400000,0.504632,0.579042,0.678571,0.093750
27,601100.SH,恒立液压,0.726252,0.897059,0.000001,0.599403,0.590909,0.055576
34,300119.SZ,瑞普生物,0.741060,0.523810,0.182721,0.841220,0.750000,0.052181
42,000955.SZ,欣龙控股,0.775282,0.800000,0.200000,0.837587,0.560674,0.116327
46,002134.SZ,天津普林,0.787823,0.500000,0.391002,0.724014,0.783333,0.012520
49,002409.SZ,雅克科技,0.702850,0.766667,0.015944,0.738777,0.933333,0.000236
51,002463.SZ,沪电股份,0.772618,0.724675,0.000203,0.746098,0.644295,0.003901
